In [24]:
import getpass
import paramiko
import os
import tarfile
import wradlib as wrl
import cv2
import numpy as np
import pickle
import time
from os.path import expanduser

In [34]:
#Decides if files should be downloaded
extractFiles = True
#Decides if files should be extracted
downloadCompleteYear = True
#Decides if files should be converted to PNGS
generatePNGs = True
#Decides if raw files should be deleted
deleteFiles = True
#Decides if the Trash should be deleted
deleteTrash = True

In [38]:
HOST = "iosds01.ios.htwg-konstanz.de"
PORT = 22

#Place of Data which should be downloaded (From Server)
#The whole Directory (Year) will be downloaded
DOWNLOAD_PATH = "./datascience/regenradar/Radardaten/2016"

#Place where the raw data should be stored. This whole directory will be DELETED, if the PNGs are generated. 
STORE_PATH_RAW = os.path.abspath("./") + "/Data/RawData"

#Place where the generated PNGs should be stored. 
STORE_PATH_PNG = os.path.abspath("./") + "/Data"

#Place where all Radaradata from one month (zip) from server should be stored
PATH_TO_RADAR_ZIP = STORE_PATH_RAW + "/RadDataZIP"

#The path of the Trash. If deleteTrash == True, this directory (Trash) will be deleted after every generated month
PATH_TO_TRASH = expanduser("~") + "/.local/share/Trash/files"

Funktionen zum downloaden der Daten

In [48]:
def printProgress(transferred, toBeTransferred):

    width = 50
    percent = transferred / toBeTransferred
    size = int(percent * width)
    fetched = transferred / (1024**2)
    total = toBeTransferred / (1024**2)
    progress = "%(current)d/%(total)d Mb"

    args = {
        "current": fetched,
        "total": total
    }
    bar = progress + ' [' + '#' * size + '#' + '.'*(width - size) + ']'
    print("\r"+bar % args, end='')

def downloadDataSet(root_dir,host,port,DOWNLOAD_PATH, STORE_PATH_RAW):

    path = root_dir
    
    transport = paramiko.Transport((host, port))
        
    print("Downloading from:", str(host + "/"+DOWNLOAD_PATH))
    username = input('Enter your username\n')
    pswd = getpass.getpass('Enter your password:\n')
    print("Connecting to "+host+"...")

    transport.connect(username=username, password=pswd)
    sftp = paramiko.SFTPClient.from_transport(transport)

    files_on_nas = sftp.listdir(DOWNLOAD_PATH)
    print(files_on_nas)
    
    
    for month in files_on_nas:    
        if(downloadCompleteYear == True): 
            if os.path.exists(path) == False:
                os.makedirs(path)
            CURRENT_DOWNLOAD_PATH = DOWNLOAD_PATH + "/" + month

            #Die Daten für den Monat downloaden
            print("Downloading...")
            sftp.get(remotepath=CURRENT_DOWNLOAD_PATH, localpath=path +
                         "/"+CURRENT_DOWNLOAD_PATH.split("/")[-1], callback=printProgress)
            print("\n")
        
        if(extractFiles == True):
            #Extract the downloaded data
            extractData("/"+month)
        if(generatePNGs == True):
            #Create the PNGs
            createPNGs(month)

        if(deleteFiles == True):
            ##################################################
            #### HERE THE RAW DATA WILL BE DELETED
            #### ATTENTION, IF THE WRONG PATH IS SET, IT COULD DELETE THE COMPLETE DISK!!!!
            ##################################################
            for root, dirs, files in os.walk(STORE_PATH_RAW, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    if(name != "MonthPNGData"):
                        os.rmdir(os.path.join(root, name))
        if(deleteTrash == True):
            #Remove Files and Directorys in Trash
            for root, dirs, files in os.walk(PATH_TO_TRASH, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))

Funktionen zum transformieren der Daten zu PNGs

In [46]:
def initTranformRadarToPng ():
    RadarValues99 = np.asanyarray(pickle.load(open("RadarValues99PercentDatapoints.p", "rb")))
    maxValue = 40 # actual maxVal = maxVal + 99% Quanil (which is 1,39)
    remaining_range = 255-len(RadarValues99)
    scaleFactor = remaining_range/maxValue
    maxOfRadarValues99 = max(RadarValues99[:,0])
    RadarValues99 = np.sort(RadarValues99[:,0])
    return RadarValues99, maxOfRadarValues99, scaleFactor
    
def transformRadarDataToPng (radarData, RadarValues99, maxOfRadarValues99, scaleFactor):
    #scale values which are larger than maxOfRadarValues99
    radarData = np.where(radarData>maxOfRadarValues99, 
                         ((radarData-maxOfRadarValues99)*scaleFactor+len(RadarValues99)).astype(int), 
                         radarData)
    #set values that are larger than 255 to 255
    radarData[radarData>255]=255

    for idx in range(1,len(RadarValues99)):
        radarData[(radarData<=RadarValues99[-idx]) & (radarData>RadarValues99[-idx-1])] = len(RadarValues99)-idx

    radarData[(radarData>0) & (radarData<RadarValues99[0])]=1
    return radarData

# Array-Like, max of all data
def normalize(data, absolute_max):
    factor = float(254)/absolute_max
    data_scaled = []
    for row in data:
        scaled_row = []
        for pixel in row:
            if pixel == 0:
                scaled_row.append(pixel)
            else:
                scaled_row.append(pixel*factor+1)
        data_scaled.append(scaled_row)

    return np.asanyarray(data_scaled)

def save_png_grayscale_8bit(image_data, filepath, filename, factor=1):
    image_data_8bit = image_data.astype(np.uint8)
    image_data_8bit *= int(factor)
    
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    full_filename = filepath + filename + ".png"
    cv2.imwrite(full_filename, image_data_8bit)
    
def get_timestamp_for_bin_filename(bin_file_name):
    split = bin_file_name.split('-')
    timestamp = split[2]
    return timestamp

def extractData(ZIP_TO_DOWNLOAD):
    #Extract the downloaded data, if extractFiles is True
    if extractFiles == True:
        tar_month = tarfile.open(PATH_TO_RADAR_ZIP + ZIP_TO_DOWNLOAD, "r:")
        tar_month.extractall(STORE_PATH_RAW + "/RadDataZipMonth")
        list_days = os.listdir(STORE_PATH_RAW + "/RadDataZipMonth")
        
        counter = 0
        for day in list_days:
            tar_month = tarfile.open(STORE_PATH_RAW + "/RadDataZipMonth/" + day, "r:gz")
            tar_month.extractall(STORE_PATH_RAW + "/MonthsRadarData")
            
            counter = counter + 1
            print("Months extracted: {0} / {1}".format(counter, len(list_days)), end="\r")
            
def createPNGs(FILE_NAME):
    #names of all bin Data in 5 min. resolution are listed 
    listOfRadarBin5MinSteps = os.listdir(STORE_PATH_RAW + "/MonthsRadarData")
    #counter to log the progress
    counter = 0
    
    RadarValues99, maxOfRadarValues99, scaleFactor = initTranformRadarToPng()
    for binData in listOfRadarBin5MinSteps: 
        startTimeOverall = time.time()
        data, attrs = wrl.io.read_radolan_composite(STORE_PATH_RAW + "/MonthsRadarData/" + binData)
#         data = normalize(data, 5)
        startTimeForTranform = time.time()
        data = transformRadarDataToPng(data,RadarValues99, maxOfRadarValues99, scaleFactor)
        endTimeForTransform = time.time() - startTimeForTranform
        
        #Store the Data in /home/till/Dokumente/Teamprojekt/TeamProjekt_Git/Data/MonthPNGData/<NameOfDownloadedZip>
        save_png_grayscale_8bit(data, STORE_PATH_PNG + "/MonthPNGData/" + FILE_NAME[:-4] + "/", get_timestamp_for_bin_filename(binData), factor=1)
        endTimeForStoreData = time.time() - startTimeForTranform - endTimeForTransform
        
        counter = counter + 1
        print("Picture {0} / {1} Time for transform: {2} Time for store: {3}".format(counter, len(listOfRadarBin5MinSteps), endTimeForTransform, endTimeForStoreData), end="\r")

In [47]:
#Download data from Server if download is True
downloadDataSet(PATH_TO_RADAR_ZIP,HOST,PORT,DOWNLOAD_PATH, STORE_PATH_RAW)

Enter your username
till_r
Enter your password:
········
Connecting to iosds01.ios.htwg-konstanz.de...
['YW2017.002_201602.tar', 'YW2017.002_201604.tar', 'YW2017.002_201601.tar', 'YW2017.002_201609.tar', 'YW2017.002_201612.tar', 'YW2017.002_201608.tar', 'YW2017.002_201605.tar', 'YW2017.002_201607.tar', 'YW2017.002_201611.tar', 'YW2017.002_201610.tar', 'YW2017.002_201603.tar', 'YW2017.002_201606.tar']
Downloading...
247/247 Mb [###################################################]



Socket exception: Connection timed out (110)


Downloading... 8640 Time for transform: 0.2330763339996338 Time for store: 0.00516915321350097665
228/228 Mb [###################################################]

Downloading... 8928 Time for transform: 0.24243545532226562 Time for store: 0.0037002563476562534
326/326 Mb [###################################################]

Downloading... 8928 Time for transform: 0.23820090293884277 Time for store: 0.0040175914764404315
393/393 Mb [###################################################]

Downloading... 8928 Time for transform: 0.2385859489440918 Time for store: 0.00390076637268066454
373/373 Mb [###################################################]

Downloading... 8928 Time for transform: 0.23937439918518066 Time for store: 0.0043532848358154336
406/406 Mb [###################################################]

Downloading... 8640 Time for transform: 0.23531675338745117 Time for store: 0.0038106441497802734
415/415 Mb [###################################################]

Downloading... 8